In [1]:
import networkx as nx

In [11]:
nx.__version__

'3.2.1'

In [7]:
INPUT_FILE_PATH = '/Users/alexander/Downloads/5_frag/main.graphml' #main graph ML input
OUTPUT_PATH = '/Users/alexander/Downloads/5_frag_t'                #output folder
OUTPUT_NAME = 'test_graph_name'                                    #output file name 

In [6]:
G = nx.read_graphml(INPUT_FILE_PATH)

In [8]:
print("number of components:", nx.number_connected_components(G))
Gcc = sorted(nx.connected_components(G), key=len, reverse=True)
Gc = G.subgraph(Gcc[0])
print("largest component: ",len(Gc))

number of components: 1177
largest component:  1344


In [10]:

print("Entire graph: %s nodes\nGCC: %s nodes\nDensity: %.4f"%(len(G.nodes()), len(Gc.nodes()),nx.density(Gc) ))

comm = nx.community.louvain_communities(Gc, seed=123)
print("Communities: %s"%len(comm))
mod_val = nx.community.modularity(Gc,comm)
print("Modularity: %.4f"%mod_val)

for ind,c in enumerate(comm):
    for i in c:
        Gc.nodes[i]['community_id'] = ind + 1


for i in Gc:
    st = True
    for n in Gc.neighbors(i):
        if Gc.edges[i,n]['cosine_score'] >= 0.7:
            st = False
            break
    if st == True:
        Gc.nodes[i]['singleton'] = '1'
    else:
        Gc.nodes[i]['singleton'] = '0'


nx.write_graphml(Gc,f'{OUTPUT_PATH}/{OUTPUT_NAME}.graphml')

Gc_copy = nx.Graph(Gc)
for i in Gc.edges(data=True):
    id1 = Gc.nodes[i[0]]['community_id']
    id2 = Gc.nodes[i[1]]['community_id']
    if id1 != id2:
        Gc_copy.remove_edge(i[0], i[1])

nx.write_graphml(Gc_copy,f'{OUTPUT_PATH}/{OUTPUT_NAME}_cut.graphml')
print('Number of components in %s_cut.graphml (validation): %s'%(OUTPUT_NAME,nx.number_connected_components(Gc_copy)))

S = [Gc.subgraph(c).copy() for c in nx.connected_components(Gc)]

St = nx.maximum_spanning_tree(Gc_copy,weight='cosine_score')
S2 = [St.subgraph(c).copy() for c in nx.connected_components(St)]
nx.write_graphml(St,f'{OUTPUT_PATH}/{OUTPUT_NAME}_spanning_tree.graphml')

St1 = nx.maximum_spanning_tree(nx.spanner(Gc_copy,5,weight='cosine_score'),weight='cosine_score')
Gc_copy = nx.Graph(Gc)
for i in Gc.edges():
    if i not in St1.edges():
        Gc_copy.remove_edge(i[0], i[1])

nx.write_graphml(Gc_copy,f'{OUTPUT_PATH}/{OUTPUT_NAME}_spanner_spanning_tree.graphml')



print("\n\n")

Entire graph: 2670 nodes
GCC: 1344 nodes
Density: 0.0072
Communities: 23
Modularity: 0.6858
Number of components in test_graph_name_cut.graphml (validation): 23



